In [1]:
from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_community.llms.tongyi import Tongyi
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
print(DASHSCOPE_API_KEY)

# qwen
qwen_llm = Tongyi(
    model='qwen2-1.5b-instruct'
)

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn
sk-01c5003340c3453b934052d737d45e01
sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [5]:
# 提示词模板与模型的传统用法

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage
)
from langchain.chains import LLMChain

human_template = "Show me the HEX code of color {color_name}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=cf_llm, prompt=chat_prompt)

chain.run("RED")

"I'd be happy to help!\n\nThe HEX code for the color RED is #FF0000. This is a bright, vibrant red color that is often used to draw attention or evoke a sense of energy and passion.\n\nWould you like to know more about HEX codes or how they're used in design?"

In [6]:
# 通过 LEL 连接提示词模板与模型

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Show me the HEX code of color {color_name}")

chain = prompt | cf_llm

chain.invoke({"color_name": "RED"})

"I'd be happy to help you with that!\n\nThe HEX code for the color RED is #FF0000. This is a bright, vibrant red color that is often used in design and art to draw attention or evoke emotions. Would you like to know more about the different shades of red or how to use HEX codes in your design work? I'd be happy to provide more information!"

In [7]:
# 添加标准输出解析

from langchain.schema.output_parser import StrOutputParser

chain = prompt | ms_llm | StrOutputParser()

chain.invoke({"color_name": "RED"})

'The hex code for the color red is #FF0000. In the RGB color model, which is used in web design and digital graphics, the values for red, green, and blue are represented by two-digit hexadecimal numbers ranging from 00 to FF. In the case of pure red, the red value is set to its maximum (FF), while the green and blue values are set to their minimum (00). This results in the hex code #FF0000, which corresponds to the vibrant and intense color red.'

## 添加函数调用
我们来给管道中的模型添加一些函数调用。注，我们并不真正调用函数，只解析出函数调用的数据。

JsonOutputFunctionsParser 用来将函数调用的回复解析为JSON格式，请参考API 文档

In [11]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


# "required": ["hex_code", "color"] 两个字段必填
functions = [
    {
        "name": "save_color_code",
        "description": "Save the HEX code of color and its name",
        "parameters": {
            "type": "object",
            "properties": {
                "hex_code": {
                    "type": "string",
                    "description": "The HEX code of the color"
                },
                "color": {
                    "type": "string",
                    "description": "The color name"
                }
            },
            "required": ["hex_code", "color"]
        }
    }
]
chain = (
        prompt
        # 调用这个functions的方法：save_color_code
        | cf_llm.bind(function_call={"name": "save_color_code"}, functions=functions)
        | JsonOutputFunctionsParser()
)

chain.invoke({"color_name": "RED"})

OutputParserException: This output parser can only be used with a chat generation.